### Step 0 : packages library

In [ ]:
library(tidyverse)
library(data.table) 
library(DESeq2)
library(FactoMineR)
library(factoextra)
library(ggrepel)
library(pheatmap)
library(RColorBrewer)
library(ggpubr)
library(clusterProfiler)
library(org.Hs.eg.db)
library(cols4all)
library(org.Mm.eg.db)
library(clusterProfiler)
library(enrichplot)
library(tidyverse)
library(msigdbr)

### Step 1 : Data input and clean

In [ ]:
### Data input
countdata = fread('./Counts/raw_counts.txt',header = T,data.table = F) |>
    column_to_rownames("Geneid")


id_trans <- fread('./Reference/g2s_vm25_gencode.txt',header = F,data.table = F) 
colnames(id_trans) <- c("geneid","symbol")
head(id_trans)
symbol <- id_trans[match(rownames(countdata),id_trans$geneid),"symbol"]
table(duplicated(symbol))
countdata = countdata |>
    aggregate(by=list(symbol), FUN=sum) |>
    column_to_rownames('Group.1')
head(countdata)

metadata = data.frame(sampleid = colnames(countdata),
                      group = rep(c("168","90"),each = 3),
                      replicate = rep(paste0("Rep",1:3),2))
head(metadata)
rownames(metadata) = metadata$sampleid

### Data check
ddsMat = DESeqDataSetFromMatrix(countData = countdata,
                                colData = metadata,
                                design = ~group)

ddsMat_rlog = rlog(ddsMat,blind = F)
dat <- as.data.frame(assay(ddsMat_rlog))
head(dat)

dat_pca = PCA(t(dat) , graph = F,ncp = 2)
pca_sample = dat_pca$ind$coord |>
    as.data.frame()|>
    dplyr::select(PC1 = 1,PC2 = 2)|>
    rownames_to_column("sampleid") |>
    left_join(metadata,by = "sampleid")
head(pca_sample)
pca_eg1 = round(dat_pca$eig[1,2],2)
pca_eg2 = round(dat_pca$eig[2,2],2)
ggplot(pca_sample,aes(PC1,PC2))+
    geom_point(aes(color = group),size = 3)+
    theme_test()+
    labs(x= paste0("PCA1:",pca_eg1,"%"),
         y= paste0("PCA2:",pca_eg2,"%"))+
    geom_text_repel(aes(label = sampleid),size = 3,show.legend = F,box.padding = unit(0.2,"lines")) 

sampleDists <- dist(t(dat))   
sampleDistMatrix <- as.matrix(sampleDists)  
colors <- colorRampPalette( rev(brewer.pal(9, "Blues")) )(255)  
pheatmap(sampleDistMatrix,
         fontsize=7,
         clustering_distance_rows=sampleDists,
         clustering_distance_cols=sampleDists,
         angle_col=45,
         col=colors)

### Data clean
countdata = countdata|>
    dplyr::select(-"301",-"303",-"326",-"331")
metadata = data.frame(sampleid = colnames(countdata),
                      group = rep(c("TAC_NC","TAC_CD63_AS1","NC","CD63_AS1"),each = 3),
                      replicate = rep(paste0("Rep",1:3),4))
### Run <Data check> again 

### Step 2 : Different experssion analysis

In [ ]:
keep <- rowSums(counts(ddsMat)) >= 1.5*ncol(countdata) 
ddsMat <- ddsMat[keep,]                                
ddsMat_result = DESeq(ddsMat) 


results_FAC = results(ddsMat_result,contrast = c("group","CD63_AS1","NC"),pAdjustMethod = "fdr",alpha = 0.05)
results_TAC = results(ddsMat_result,contrast = c("group","TAC_CD63_AS1","TAC_NC"),pAdjustMethod = "fdr",alpha = 0.05)
results_NC = results(ddsMat_result,contrast = c("group","TAC_NC","NC"),pAdjustMethod = "fdr",alpha = 0.05)
results_CD63 = results(ddsMat_result,contrast = c("group","TAC_CD63_AS1","CD63_AS1"),pAdjustMethod = "fdr",alpha = 0.05)

write.csv(as.data.frame(results_FAC),file = "results_CD63_AS1_VS_NC.csv",quote = F)
write.csv(as.data.frame(results_TAC),file = "results_TAC_CD63_AS1_VS_TAC_NC.csv",quote = F)
write.csv(as.data.frame(results_NC),file = "results_TAC_NC_VS_NC.csv",quote = F)
write.csv(as.data.frame(results_CD63),file = "results_TAC_CD63_AS1_VS_CD63_AS1.csv",quote = F)

save(metadata,countdata,ddsMat,dat,results_FAC,results_TAC,results_NC,results_CD63,file = "RNA_seq_workflow.Rdata")

### Step 3 : Visualization

In [ ]:
result = results_CD63 |>
    as.data.frame()|>
    na.omit()|>
    rownames_to_column("symbol")
chr = "TAC_CD63_AS1"
chr_nc = "CD63_AS1"

Vacanol_plot = function(result,fc_cutoff,p_adj,p_cutoff){
if(p_adj == 1){
result_cut = result |>
    mutate(label = ifelse(abs(log2FoldChange)>=fc_cutoff & padj<=p_cutoff,symbol,""))

plot = ggplot(result_cut,aes(log2FoldChange, -log10(padj)))+
  geom_hline(yintercept = -log10(p_cutoff), linetype = "dashed", color = "#999999")+
  geom_vline(xintercept = c(-fc_cutoff,fc_cutoff), linetype = "dashed", color = "#999999")+
  geom_point(aes(size=-log10(padj), color= -log10(padj)))+
  scale_color_gradientn(values = seq(0,1,0.2),
                        colors = c("#39489f","#39bbec","#f9ed36","#f38466","#b81f25"))+
  scale_size_continuous(range = c(1,2))+
  guides(col = guide_colourbar(title = "-Log10(padj)"),
         size = "none")+
  theme_test()+
  ggtitle(paste0("Vocanol plot of ",chr," VS ",chr_nc))+
  theme(plot.title = element_text(hjust = 0.5))+
  geom_text_repel(max.overlaps = getOption("ggrepel.max.overlaps", default = 50),aes(label=label),size = 2.5,color = 'black')+
  xlab("log2FoldChange")+
  ylab("-log10(padj)")
}else{
result_cut = result |>
    mutate(label = ifelse(abs(log2FoldChange)>=fc_cutoff & pvalue<=p_cutoff,symbol,""))
plot = ggplot(result_cut,aes(log2FoldChange, -log10(pvalue)))+
  geom_hline(yintercept = -log10(p_cutoff), linetype = "dashed", color = "#999999")+
  geom_vline(xintercept = c(-fc_cutoff,fc_cutoff), linetype = "dashed", color = "#999999")+
  geom_point(aes(size=-log10(pvalue), color= -log10(pvalue)))+
  scale_color_gradientn(values = seq(0,1,0.2),
                        colors = c("#39489f","#39bbec","#f9ed36","#f38466","#b81f25"))+
  scale_size_continuous(range = c(1,2))+
  guides(col = guide_colourbar(title = "-Log10(pvalue)"),
         size = "none")+
  theme_test()+
  ggtitle(paste0("Vocanol plot of ",chr," VS ",chr_nc))+
  theme(plot.title = element_text(hjust = 0.5))+
  geom_text_repel(max.overlaps = getOption("ggrepel.max.overlaps", default = 50),aes(label=label),size = 2.5,color = 'black')+
  xlab("log2FoldChange")+
  ylab("-log10(pvalue)")
}
return(plot)}


library(readxl)
mito_gene = read_xls("./Human.MitoCarta3.0.xls",sheet = 2)
str(mito_gene)
table(mito_gene$Symbol %in% result$symbol)
result_mito = result[result$symbol %in% mito_gene$Symbol,]
Vacanol_plot(result,2,0,0.05)+geom_point(data = result_mito,aes(log2FoldChange, -log10(pvalue)),color = "red")

### Step 4 : KEGG and GO enrichment

In [ ]:
load("RNA_seq_workflow.Rdata")
gene_up = result|>
    as.data.frame()|>
    filter(log2FoldChange >= 1&pvalue<= 0.05) |>
    pull(symbol)|>
    bitr(fromType="SYMBOL",toType="ENTREZID", OrgDb = org.Hs.eg.db) |>
    pull(ENTREZID) 
gene_down = result|>
    as.data.frame()|>
    filter(log2FoldChange <= -1&pvalue<= 0.05) |>
    pull(symbol)|>
    bitr(fromType="SYMBOL",toType="ENTREZID", OrgDb = org.Hs.eg.db) |>
    pull(ENTREZID)


GO_plot = function(gene){
go_result <- enrichGO(gene = gene,
                   OrgDb=org.Hs.eg.db,
                   keyType = "ENTREZID",
                   ont = "ALL",
                   pAdjustMethod = "BH",
                   minGSSize = 1,
                   pvalueCutoff = 0.99,
                   qvalueCutoff = 0.99,
                   readable = TRUE)
go_result_df =  go_result@result |>
    group_by(ONTOLOGY) |>
    arrange(p.adjust) |>
    slice_head(n=10) |>
    ungroup()
go_result_df = go_result_df[go_result_df$ONTOLOGY %in% c("BP","CC","MF"),]

head(go_result_df)
mytheme <- theme(
axis.title = element_text(size = 13),
axis.text = element_text(size = 8),
plot.title = element_text(size = 14,
hjust = 0.5,
face = "bold"),
legend.title = element_text(size = 13),
legend.text = element_text(size = 11),
plot.margin = margin(t = 5.5,r = 10,l = 5.5,b = 5.5)
)
plot = ggplot(go_result_df, aes(Description, Count)) +
  geom_col(aes(fill = ONTOLOGY), width = 0.1) +
  geom_point(aes(size = -log10(p.adjust), color = ONTOLOGY)) +
  coord_flip() +
  labs(x = "", y = "Numbers of genes") +
  scale_color_manual(values = c(
    "#852f88",
    "#eb990c",
    "#0f8096"
  )) +
  scale_fill_manual(values = c(
    "#852f88",
    "#eb990c",
    "#0f8096"
  )) +
  scale_size_continuous(range = c(2, 8)) +
  theme_test()+mytheme+scale_x_discrete(labels = function(x) str_wrap(x, width = 70))+
  facet_grid(ONTOLOGY~.,scales = "free")+theme(strip.background = element_blank(),strip.text = element_blank())
return(plot)}
KEGG_plot = function(up,down){
kk.up <- enrichKEGG(
  gene = up,
  organism = "hsa",#"mmu"
  pvalueCutoff = 0.99,
  qvalueCutoff = 0.1
)

kk.down <- enrichKEGG(
  gene = down,
  organism = "hsa",#"mmu"
  pvalueCutoff = 0.99,
  qvalueCutoff = 0.1
)

kegg_result_df <- bind_rows(
  kk.up@result %>% filter(pvalue <= 0.05) %>% mutate(group = "up", groupno = 1),
  kk.down@result %>% filter(pvalue <= 0.05) %>% mutate(group = "down", groupno = -1)
) %>%
  group_by(group) %>%
  slice_head(n=15) %>%
  ungroup() %>%
  mutate(pvalue = -log10(pvalue), `-log10pvalue` = pvalue * groupno)|>
  mutate(Count = Count* groupno)

kegg_result_df$Description = gsub(" - Mus musculus \\(house mouse\\)", "",kegg_result_df$Description)

head(kegg_result_df)
level <- kegg_result_df |>
  filter(group %in% c('up', 'down')) |>
  pull(Description)|>
  unique()|>
  rev()
kegg_result_df$Description <- factor(kegg_result_df$Description, levels = level)

plot = ggplot(kegg_result_df,aes(x = Count,y = Description,fill = `-log10pvalue`)) +
  geom_col() +
  theme_classic()+
  scale_x_continuous(breaks = seq(-15,10,by = 5),
              labels = abs(seq(-15,10,by = 5))) +
  scale_fill_continuous_c4a_div('sunset', mid = 0,reverse = T,breaks = c(2,0,-4,-8),label = c("2","0","4","8"))+
  labs(x="Number of Genes",y = "")+
  ggtitle("KEGG pathway enrichment")+
  theme(
    plot.title = element_text(size = 14,hjust = 0.5,face = "bold"),
    axis.text = element_text(size = 12), 
    axis.title.x = element_text(size = 13),
    legend.title = element_text(size = 13), 
    legend.text = element_text(size = 12))
return(plot)}

GO_plot(gene_up)+ggtitle("GO enrichment of up_genes")
GO_plot(gene_down)+ggtitle("GO enrichment of down_genes")
KEGG_plot(gene_up,gene_down)

### Step 5 : GSEA---Gene Set Enrichment Analysis

In [ ]:
GSEA_all =function(result_deseq){
GSEA_data = result_deseq |>
  as.data.frame()
GSEA_data$entrez = mapIds(x=org.Mm.eg.db,
                          keys = row.names(GSEA_data),
                          column = "ENTREZID",
                          keytype = "SYMBOL",
                          multiVals = "first")
genelist_df = GSEA_data |>
  as.data.frame()|>
  na.omit()|>
  arrange(desc(log2FoldChange))
head(genelist_df)
genelist = genelist_df |>
  pull(log2FoldChange) 
names(genelist) = genelist_df$entrez

gsea_res_KEGG <- gseKEGG(geneList = genelist,
                         organism  = "mmu", 
                         pvalueCutoff = 1)  
gsea_res_KEGG <- DOSE::setReadable(gsea_res_KEGG, 
                             OrgDb=org.Mm.eg.db,
                             keyType='ENTREZID')

gsea_res_GO <- gseGO(geneList     = genelist,
                      ont          = "ALL", 
                      OrgDb        = org.Mm.eg.db,
                      keyType      = "ENTREZID",
                      pvalueCutoff = 1)   
gsea_res_GO  <- DOSE::setReadable(gsea_res_GO , 
                           OrgDb = org.Mm.eg.db,
                           keyType='ENTREZID')
plot_KEGG = ridgeplot(gsea_res_KEGG,
          showCategory = 20,
          fill = "pvalue", 
          core_enrichment = TRUE,
          label_format = 40,
          orderBy = "NES",
          decreasing = FALSE
          )+
  theme_test()+
  ggtitle("GSEA result of KEGG")+
  labs(x= "Normalized Enrichment Score")+
  theme(axis.text.y = element_text(size=8),
        plot.title = element_text(hjust = 0.5))

plot_GO = ridgeplot(gsea_res_GO,
          showCategory = 20,
          fill = "pvalue", 
          core_enrichment = TRUE,
          label_format = 40,
          orderBy = "NES",
          decreasing = FALSE
          )+
  theme_test()+
  ggtitle("GSEA result of GO")+
  labs(x= "Normalized Enrichment Score")+
  theme(axis.text.y = element_text(size=8),
        plot.title = element_text(hjust = 0.5))
plot_all = list(KEGG = plot_KEGG,GO = plot_GO)
return(plot_all)}

GSEA_all(results_CD63)

### Step 6 heatmap multiple groups 

In [ ]:
load("./RNA_seq_workflow.Rdata")
head(results_NC)
head(results_TAC)

results_NC_df = results_NC |>
    as.data.frame() |>
    na.omit() |>
    dplyr::select(log2FoldChange,pvalue)|>
    dplyr::rename("log2FC_NC" = "log2FoldChange","pvalue_NC" = "pvalue")|>
    rownames_to_column("gene_name")
results_TAC_df = results_TAC |>
    as.data.frame() |>
    na.omit() |>
    dplyr::select(log2FoldChange,pvalue)|>
    dplyr::rename("log2FC_TAC" = "log2FoldChange","pvalue_TAC" = "pvalue")|>
    rownames_to_column("gene_name")

result_heatmap = merge(results_TAC_df,results_NC_df,by = "gene_name")
head(result_heatmap)
result_heatmap =result_heatmap |>
    mutate(logFC_all = -log2FC_TAC+log2FC_NC,pvalue_all = -log10(pvalue_TAC)-log10(pvalue_NC)) |>
    arrange(desc(pvalue_all),desc(logFC_all))|>
    head(50)
head(dat)
ann_colors = list(group = c(TAC_NC = "darkred", NC = "forestgreen",TAC_CD63_AS1 = "darkorange"))
choose_compare = metadata[1:9,]
Top_M = dat[result_heatmap$gene_name,choose_compare$sampleid] 
head(Top_M)
annotation_col = choose_compare |>
    column_to_rownames("sampleid") |>
    dplyr::select(group)
head(annotation_col)
Top_M = Top_M[,c("325","327","330","298","300","302","304","307","308")]
pheatmap(Top_M, 
         color = colorRampPalette(rev(brewer.pal(n = 7, name = "RdYlBu")))(100), 
         scale = "row", 
         annotation_col = annotation_col, 
         annotation_colors = ann_colors,
         fontsize = 12,
         fontsize_row =9, 
         cellwidth = 55, 
         show_colnames = T,
         cluster_cols = F,
         angle_col= "45")
#### use more functional packages
library(Mfuzz)
library(ClusterGVis)
head(dat)
dat_sub = dat[,c("325","327","330","298","300","302","304","307","308")]

dat_group <- as.data.frame(t(dat_sub)) |>
    mutate(group = rep(c("NC","TAC_NA","TAC_CD63_AS1"),each = 3)) |>
    group_by(group)|>
    mutate(across(where(is.numeric), mean)) |>
    ungroup()|>
    unique() |>
    column_to_rownames("group") |>
    t()
head(dat_group)
class(dat_group)
dat_group <- dat_group |>
  as.data.frame()|>
  rownames_to_column("gene_name")|>
  rowwise() |>
  mutate(sd = sd(c_across(where(is.numeric))))|>
  arrange(desc(sd)) |>
  head(5000)
dat_group = dat_group |>
  as.data.frame()|>
  column_to_rownames("gene_name") |>
  dplyr::select(-sd) 
head(dat_group)
dat_group =as.matrix(dat_group)
getClusters(exp = dat_group)
cm <- clusterData(exp = dat_group,
                  cluster.method = "mfuzz",
                  cluster.num = 6)
markGenes = c("Myh7","Nppa","Myh2","Nppb")
enrich <- enrichCluster(object = cm,
                        OrgDb = org.Mm.eg.db,
                        type = "BP",
                        pvalueCutoff = 0.05,
                        topn = 5,
                        seed = 5201314)
visCluster(object = cm,
           plot.type = "both",
           ht.col.list =  list(col_range = c(-1.5, 0, 1.5),col_color = c("#08519C", "white", "#A50F15")),
           column_names_rot = 45,
           show_row_dend = F,
           markGenes = markGenes,
           markGenes.side = "left",
           genes.gp = c('italic',fontsize = 12,col = "black"),
           annoTerm.data = enrich,
           line.side = "left",
           go.col = rep(ggsci::pal_d3()(6),each = 5),
           go.size = "pval",
           add.bar = T,
           textbar.pos = c(0.9,1.2)
          #  subgroup.anno = c("C4","C6","C8")
          )

str(cm)
head(cm$wide.res)
cm_cluster6 = subset(cm$wide.res,cluster == 6)
head(cm_cluster6)
cm_cluster6$description = mapIds(x=org.Mm.eg.db,
                                      keys = cm_cluster6$gene,
                                      column = "GENENAME",
                                      keytype = "SYMBOL",
                                      multiVals = "first")
write.csv(cm_cluster6,file= "cm_cluster6.csv")